In [1]:
import os
import pickle
import collections
import datetime
import copy
import numpy as np
import scipy as sp
import pandas as pd
import tensorflow as tf
import sklearn.model_selection

In [2]:
data_dir = r"/mnt1/zhaodachuan/data/predict_user_attribute/age/raw_data"

In [3]:
file_list = os.listdir(data_dir)
user_files = []
article_files = []
for file_name in file_list:
    if 'user_' in file_name:
        user_files.append(file_name)
    elif 'part-' in file_name:
        article_files.append(file_name)
    else:
        pass
user_files.sort()
article_files.sort()
print(user_files)
print(article_files)

['user_new']
['part-00000', 'part-00001', 'part-00002', 'part-00003', 'part-00004', 'part-00005', 'part-00006', 'part-00007', 'part-00008', 'part-00009', 'part-00010', 'part-00011', 'part-00012', 'part-00013', 'part-00014', 'part-00015', 'part-00016', 'part-00017', 'part-00018', 'part-00019', 'part-00020', 'part-00021', 'part-00022', 'part-00023', 'part-00024', 'part-00025', 'part-00026', 'part-00027', 'part-00028', 'part-00029', 'part-00030', 'part-00031', 'part-00032', 'part-00033', 'part-00034', 'part-00035', 'part-00036', 'part-00037', 'part-00038', 'part-00039', 'part-00040', 'part-00041', 'part-00042', 'part-00043', 'part-00044', 'part-00045', 'part-00046', 'part-00047', 'part-00048', 'part-00049', 'part-00050', 'part-00051', 'part-00052', 'part-00053', 'part-00054', 'part-00055', 'part-00056', 'part-00057', 'part-00058', 'part-00059', 'part-00060', 'part-00061', 'part-00062', 'part-00063', 'part-00064', 'part-00065', 'part-00066', 'part-00067', 'part-00068', 'part-00069', 'part-

In [4]:
%%time
#读取用户数据
all_file_data = []
for file_name in user_files:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None))
user_table = pd.concat(all_file_data,axis=0,ignore_index=True)

user_table.columns = ['用户id','用户姓名','用户性别','用户出生时间','所在公司','所处职位','行业','微信网名','微信性别','常驻省','常驻市','是否付费用户','付费等级','关注的作者','作者类型','关注的马甲']
# 获得指定列
useful_user_df = user_table[["用户id","用户姓名","用户出生时间","微信网名","关注的作者","关注的马甲"]]
del all_file_data
del user_table
useful_user_df

CPU times: user 3.65 s, sys: 217 ms, total: 3.87 s
Wall time: 3.86 s


,用户id,用户姓名,用户出生时间,微信网名,关注的作者,关注的马甲
0,00001692e9a0e16ce869b88edaf023a9db8e61d0,-1,0-0-0,-1,"1534653,1870542","华夏基石e洞察,caoz的梦呓©"
1,00001D26FE991A4BC4A01325D5E222DE,0,-1,"0,1",-1,-1
2,00008225fde7a2d26dd0e7b770cb6de22e92654d,-1,0-0-0,-1,-1,-1
3,0000BE70BB0233665552E7C1AC45DDA7,0,-1,"0,1",-1,-1
4,0000d2529e07eb6355a786fa03d7db67f72cccc6,0,-1,"0,1",-1,-1
...,...,...,...,...,...,...
2738209,qlu2hw9gd45m34i3zgyd,0,-1,"0,1",-1,-1
2738210,qlu2mrq87mtpyzerr0s,0,-1,"0,1",-1,-1
2738211,qlu2n729ejbhpp59tsq,0,-1,"0,1",-1,-1
2738212,qlu2xlbspo1bw8wwujh,-1,1-1-1,"0,1",-1,-1


In [5]:
def deal_with_birth_date(birth_date):
    if (not birth_date.split("-")[0]) or (int(birth_date.split("-")[0]) < 1940):
        return False
    return True

useful_user_df = useful_user_df[useful_user_df["用户出生时间"].apply(deal_with_birth_date)]

In [6]:
%%time
#读取文章数据
all_file_data = []
for file_name in article_files:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None))
article_table = pd.concat(all_file_data,axis=0,ignore_index=True)

article_table.columns = ['用户id','文章id','是否精选文章','浏览文章时间','浏览时长','文章频道','文章类型','文章标签','文章图片数量','文章字数','新定位省','新定位市','评论内容','打赏金额','设备型号','使用网络']
# 获得指定列
useful_article_data = article_table[['用户id','文章频道','浏览文章时间','文章类型','文章图片数量','文章字数']]
del all_file_data
del article_table
useful_article_data

CPU times: user 53.8 s, sys: 2.5 s, total: 56.3 s
Wall time: 56.3 s


,用户id,文章频道,浏览文章时间,文章类型,文章图片数量,文章字数
0,002947a7781e89bffdafcad888898e2a73594747,"-1,金融地产,十亿消费者,财经,财经,金融地产,十亿消费者","2020-07-22 17:21:01,2020-07-16 08:35:38,2020-0...","-1,乐视","-1,1","-1,<p><span class=""text-remarks"" label=""备注"">本文..."
1,00297F3B00C4CD076C22FA29796929D1,"-1,文化教育,生活腔调,生活腔调,文化教育","2020-06-20 09:46:13,2020-06-20 09:33:22,2020-0...",-1,"-1,2","<p><span class=""text-remarks"" label=""备注"">本文来自微..."
2,006431b78c728426b28ed16371ee105b5aad5adf,-1,"2020-07-19 13:10:39,2020-06-30 10:26:58,2020-0...",-1,-1,-1
3,006A3DEA36EB70939ED2E958BE24090A,"-1,全球热点,出海,娱乐淘金,社交通讯,社交通讯,娱乐淘金,年轻一代","2020-07-14 12:20:38,2020-06-24 10:38:19,2020-0...","-1,创投,大公司,软件应用,航空航天","17,-1,3,12,5,8","-1,<p><span class=""text-remarks"" label=""备注""></..."
4,00A099F603433E280303E4253B501BE9,"十亿消费者,-1","2020-06-07 11:48:26,2020-06-07 11:43:16,2020-0...","-1,商业模式","13,-1,14","-1,<p><span class=""text-remarks"" label=""备注"">本文..."
...,...,...,...,...,...,...
203997,phojzwrkcm2ck4i9rw4w,"-1,医疗健康","2020-07-23 15:52:01,2020-07-29 18:14:29,2020-0...","军事政治,空,-1","1,9,-1","-1,<p>一直以来，人大代表建议和政协委员提案都是每年两会的重要成果，亦是传递民众关切、吸..."
203998,ppnlvxat3q36kbnbc3wa,-1,"2020-06-16 04:03:55,2020-06-16 04:03:54,2020-0...","商业模式,金融其他,-1","1,8,11,7,-1",<p>&nbsp;7年来，生鲜零售领域城头变幻大王旗，你方唱罢我方登场，两三年就换一波新的挑...
203999,q2lo39n54la38fftf6,-1,"2020-07-31 09:13:32,2020-07-31 09:12:57",-1,-1,-1
204000,q6jcef3gusgdf8tghfu,-1,2020-07-27 09:36:44,-1,-1,-1


In [7]:
user_df = useful_user_df.merge(useful_article_data,how="left")
del useful_user_df
del useful_article_data

In [8]:
user_df

,用户id,用户姓名,用户出生时间,微信网名,关注的作者,关注的马甲,文章频道,浏览文章时间,文章类型,文章图片数量,文章字数
0,007DFA864B32EBC0280192C72438DB89,春秋,1991-9-26,-1,2061955,我不叫塞尔达,"-1,全球热点,出海,年轻一代","2020-06-22 08:06:42,2020-06-02 11:17:35,2020-0...","年轻人,-1,软件应用","17,-1,7","-1,<p><span class=""text-remarks"" label=""备注"">出品..."
1,00B4D09FC36D629E3249D55B59813BB3,贾博鑫,1985-11-24,hyacinthusboy,1342281,周超臣,-1,"2020-07-30 08:56:57,2020-07-25 11:36:05,2020-0...",-1,-1,-1
2,00E80E5F9B2EF6EDA96AD73A3257B27E,-1,1990-11-14,-1,-1,-1,-1,"2020-07-04 18:29:10,2020-07-04 18:31:00,2020-0...",-1,-1,-1
3,00ECCE733FA3664C620DD1515DC24D78,-1,1983-6-1,jun0195,"1390243,456826,456836,2392109,456869,2061955,4...","吴倩男,风向·标,吸睛收购案,钛禾产业观察,-1,我不叫塞尔达,互联网指北,区块链动态,侯宏...","-1,金融地产,年轻一代,娱乐淘金,医疗健康,文化教育,前沿科技,娱乐淘金,年轻一代","2020-06-03 23:25:50,2020-07-03 22:35:57,2020-0...","-1,医疗,大公司,航空航天","-1,1,21,2,7,6","<p><span class=""text-remarks"" label=""备注"">出品丨虎嗅..."
4,013795e3bf12565ea7f8c179806e633d5965664e,海深蓝,1985-1-7,-1,"456875,500260,500259","百度又有什么新动作,头条表示不服,美团在干嘛？",-1,"2020-06-19 18:36:03,2020-07-06 21:04:14,2020-0...",-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
2211,qiwgvftlwg277kr6fc,-1,1991-3-8,"0,1,-1",-1,-1,-1,2020-07-29 10:13:10,-1,-1,-1
2212,qjswu7xcqfs2blyua7h,扶嵩,1983-1-1,"0,1,-1",-1,-1,-1,"2020-07-28 14:28:11,2020-07-28 14:05:30,2020-0...",-1,-1,-1
2213,qk2lzj8owr81hz81fan8,-1,1993-12-18,ydyd51,-1,-1,-1,"2020-07-25 09:22:21,2020-07-25 09:41:57,2020-0...",-1,"-1,1","<div><font face=""楷体"">一个鞋类电商的成长史。从儿时创办蚯蚓养殖场到大学经..."
2214,qkil95x7vd15adb5a4lz,吴存志,1978-8-12,-1,-1,-1,-1,"2020-07-23 13:00:05,2020-07-23 13:02:35,2020-0...",-1,-1,-1


In [9]:
user_df.fillna("",inplace=True)

In [10]:
def temp_function(time_str):
    if not time_str:
        return ""
    time_str_split = time_str.split(",")
    hour_list = []
    for _time in time_str_split:
        hour_list.append(str(datetime.datetime.strptime(_time, '%Y-%m-%d %H:%M:%S').hour))
    return ",".join(hour_list)
user_df["浏览文章时间"] = user_df["浏览文章时间"].apply(lambda x : temp_function(x))

In [11]:
user_df

,用户id,用户姓名,用户出生时间,微信网名,关注的作者,关注的马甲,文章频道,浏览文章时间,文章类型,文章图片数量,文章字数
0,007DFA864B32EBC0280192C72438DB89,春秋,1991-9-26,-1,2061955,我不叫塞尔达,"-1,全球热点,出海,年轻一代","8,11,16,9,19,8,18,17,14,21","年轻人,-1,软件应用","17,-1,7","-1,<p><span class=""text-remarks"" label=""备注"">出品..."
1,00B4D09FC36D629E3249D55B59813BB3,贾博鑫,1985-11-24,hyacinthusboy,1342281,周超臣,-1,"8,11,8,13,8,23,11,20,18,15,12",-1,-1,-1
2,00E80E5F9B2EF6EDA96AD73A3257B27E,-1,1990-11-14,-1,-1,-1,-1,"18,18,18,18,18,18",-1,-1,-1
3,00ECCE733FA3664C620DD1515DC24D78,-1,1983-6-1,jun0195,"1390243,456826,456836,2392109,456869,2061955,4...","吴倩男,风向·标,吸睛收购案,钛禾产业观察,-1,我不叫塞尔达,互联网指北,区块链动态,侯宏...","-1,金融地产,年轻一代,娱乐淘金,医疗健康,文化教育,前沿科技,娱乐淘金,年轻一代","23,22,23,12,19,6,19,18,8,6,7,13,16,22,6,17,12,...","-1,医疗,大公司,航空航天","-1,1,21,2,7,6","<p><span class=""text-remarks"" label=""备注"">出品丨虎嗅..."
4,013795e3bf12565ea7f8c179806e633d5965664e,海深蓝,1985-1-7,-1,"456875,500260,500259","百度又有什么新动作,头条表示不服,美团在干嘛？",-1,"18,21,8,9,21,21,8",-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
2211,qiwgvftlwg277kr6fc,-1,1991-3-8,"0,1,-1",-1,-1,-1,10,-1,-1,-1
2212,qjswu7xcqfs2blyua7h,扶嵩,1983-1-1,"0,1,-1",-1,-1,-1,"14,14,14,16,19,14,14,14",-1,-1,-1
2213,qk2lzj8owr81hz81fan8,-1,1993-12-18,ydyd51,-1,-1,-1,"9,9,9,9,9",-1,"-1,1","<div><font face=""楷体"">一个鞋类电商的成长史。从儿时创办蚯蚓养殖场到大学经..."
2214,qkil95x7vd15adb5a4lz,吴存志,1978-8-12,-1,-1,-1,-1,"13,13,13",-1,-1,-1


## 数据分析

In [ ]:
temp_list = []
for i in user_df['用户出生时间']:
    if (not i.split("-")[0]) or (int(i.split("-")[0]) < 1940):
        temp_list.append(i)
len(temp_list)

In [ ]:
#查看信息
user_df.info()

In [ ]:
user_df['用户姓名'].unique()

In [ ]:
(user_df['用户姓名']=="-1").sum()

In [ ]:
#男，女，默认，缺失 -1
user_df['用户出生时间'].unique()

In [ ]:
user_df['微信网名'].unique()

In [ ]:
user_df['关注的作者'].unique()

In [ ]:
(user_df['关注的作者']=="-1").sum()

In [ ]:
temp_list = []
for _str in user_df['关注的作者'].fillna(""):
    for _id in _str.split(","):
        temp_list.append(_id)

In [ ]:
collections.Counter(temp_list)

In [ ]:
#查看行业信息
user_df['关注的马甲'].unique()

In [ ]:
(user_df['关注的马甲']=="-1").sum()

In [ ]:
user_df["文章频道"].unique()

In [ ]:
temp_list = []
for _str in user_df["文章频道"].fillna(""):
    for v in _str.split(","):
        temp_list.append(v)
collections.Counter(temp_list)

In [ ]:
temp_list = []
for _str in user_df["文章类型"].fillna(""):
    for v in _str.split(","):
        temp_list.append(v)
collections.Counter(temp_list)

In [ ]:
user_df.fillna("",inplace=True)

In [ ]:
user_df["关注的作者"]
user_df["关注的马甲"]
user_df["文章频道"]
user_df["浏览文章时间"]
user_df["文章类型"]
user_df["文章图片数量"]
user_df["文章字数"]

In [ ]:
user_df

In [ ]:
user_df["文章字数"][0]

In [145]:
def temp_function(time_str):
    if not time_str:
        return ""
    time_str_split = time_str.split(",")
    hour_list = []
    for _time in time_str_split:
        hour_list.append(datetime.datetime.strptime(_time, '%Y-%m-%d %H:%M:%S').hour)
    return hour_list
user_df["浏览文章时间"] = user_df["浏览文章时间"].apply(lambda x : temp_function(x))

0                   [8, 11, 16, 9, 19, 8, 18, 17, 14, 21]
1               [8, 11, 8, 13, 8, 23, 11, 20, 18, 15, 12]
2                                [18, 18, 18, 18, 18, 18]
3       [23, 22, 23, 12, 19, 6, 19, 18, 8, 6, 7, 13, 1...
4                               [18, 21, 8, 9, 21, 21, 8]
                              ...                        
2211                                                 [10]
2212                     [14, 14, 14, 16, 19, 14, 14, 14]
2213                                      [9, 9, 9, 9, 9]
2214                                         [13, 13, 13]
2215                                         [12, 10, 10]
Name: 浏览文章时间, Length: 2216, dtype: object

In [ ]:
user_df.info()

# 模型训练

In [12]:
user_df_copy = copy.deepcopy(user_df)

In [13]:
user_df = copy.deepcopy(user_df_copy)
user_df

,用户id,用户姓名,用户出生时间,微信网名,关注的作者,关注的马甲,文章频道,浏览文章时间,文章类型,文章图片数量,文章字数
0,007DFA864B32EBC0280192C72438DB89,春秋,1991-9-26,-1,2061955,我不叫塞尔达,"-1,全球热点,出海,年轻一代","8,11,16,9,19,8,18,17,14,21","年轻人,-1,软件应用","17,-1,7","-1,<p><span class=""text-remarks"" label=""备注"">出品..."
1,00B4D09FC36D629E3249D55B59813BB3,贾博鑫,1985-11-24,hyacinthusboy,1342281,周超臣,-1,"8,11,8,13,8,23,11,20,18,15,12",-1,-1,-1
2,00E80E5F9B2EF6EDA96AD73A3257B27E,-1,1990-11-14,-1,-1,-1,-1,"18,18,18,18,18,18",-1,-1,-1
3,00ECCE733FA3664C620DD1515DC24D78,-1,1983-6-1,jun0195,"1390243,456826,456836,2392109,456869,2061955,4...","吴倩男,风向·标,吸睛收购案,钛禾产业观察,-1,我不叫塞尔达,互联网指北,区块链动态,侯宏...","-1,金融地产,年轻一代,娱乐淘金,医疗健康,文化教育,前沿科技,娱乐淘金,年轻一代","23,22,23,12,19,6,19,18,8,6,7,13,16,22,6,17,12,...","-1,医疗,大公司,航空航天","-1,1,21,2,7,6","<p><span class=""text-remarks"" label=""备注"">出品丨虎嗅..."
4,013795e3bf12565ea7f8c179806e633d5965664e,海深蓝,1985-1-7,-1,"456875,500260,500259","百度又有什么新动作,头条表示不服,美团在干嘛？",-1,"18,21,8,9,21,21,8",-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
2211,qiwgvftlwg277kr6fc,-1,1991-3-8,"0,1,-1",-1,-1,-1,10,-1,-1,-1
2212,qjswu7xcqfs2blyua7h,扶嵩,1983-1-1,"0,1,-1",-1,-1,-1,"14,14,14,16,19,14,14,14",-1,-1,-1
2213,qk2lzj8owr81hz81fan8,-1,1993-12-18,ydyd51,-1,-1,-1,"9,9,9,9,9",-1,"-1,1","<div><font face=""楷体"">一个鞋类电商的成长史。从儿时创办蚯蚓养殖场到大学经..."
2214,qkil95x7vd15adb5a4lz,吴存志,1978-8-12,-1,-1,-1,-1,"13,13,13",-1,-1,-1


In [15]:
multi_hot_column_odict = collections.OrderedDict()
multi_hot_column_set = set(["关注的作者","关注的马甲","文章频道","浏览文章时间","文章类型","文章图片数量"])
for i,column in enumerate(multi_hot_column_set):
    multi_hot_column_odict[column] = sklearn.preprocessing.MultiLabelBinarizer()
    split_column_list = [item.split(",") for item in user_df[column]]
    user_df[column] = multi_hot_column_odict[column].fit_transform(split_column_list).tolist()

y_col = '用户出生时间'
multi_hot_column_set.add(y_col)
user_df =  user_df[multi_hot_column_set]

In [16]:
# 1-20岁
# 20-25岁
# 26-29岁
# 30-35岁
# 40-45岁
# 46-50岁
# 50以上
def bucket_age_ont_hot(birth_date_str):
    age = datetime.datetime.now().year - int(birth_date_str.split("-")[0])
    category = 7
    y_list = [0 for _ in range(category)]
    if age < 20:
        y_list[0] = 1
    elif age < 25:
        y_list[1] = 1
    elif age < 29:
        y_list[2] = 1
    elif age < 35:
        y_list[3] = 1
    elif age < 45:
        y_list[4] = 1
    elif age < 50:
        y_list[5] = 1
    else:
        y_list[6] = 1
    return y_list

def bucket_age(birth_date_str):
    age = datetime.datetime.now().year - int(birth_date_str.split("-")[0])
    if age < 20:
        return 0
    elif age < 25:
        return 1
    elif age < 29:
        return 2
    elif age < 35:
        return 3
    elif age < 45:
        return 4
    elif age < 50:
        return 5
    else:
        return 6

user_df[y_col] = user_df[y_col].apply(lambda x:bucket_age_ont_hot(x))

/home/zhaodachuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [117]:
train, test = sklearn.model_selection.train_test_split(user_df, test_size=0.2)
train, val = sklearn.model_selection.train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

1417 train examples
355 validation examples
444 test examples


In [118]:
def generate_xy(data_df, y_col, multi_hot_column_odict, shuffle=True):
    """"""
    x_df = data_df.copy()
    x_list = []
    y_list = []
    for index in x_df.index:
        temp_x_list = np.concatenate(x_df.loc[index,[column for column in multi_hot_column_odict]]).tolist()
        temp_y_list = x_df.loc[index,y_col]
        x_list.append(temp_x_list)
        y_list.append(temp_y_list)
    return np.array(x_list,dtype=np.float32), np.array(y_list,dtype=np.int32)

In [125]:
%%time
x_train,y_train = generate_dataset(train,y_col,multi_hot_column_odict)
x_vali,y_valid = generate_dataset(val,y_col,multi_hot_column_odict)
x_test,y_test = generate_dataset(test,y_col,multi_hot_column_odict)

CPU times: user 2.47 s, sys: 7.57 ms, total: 2.48 s
Wall time: 2.45 s


In [126]:
'关注的作者'
'关注的马甲'
'文章频道'
'浏览文章时间'
'文章类型'
'文章图片数量'

'文章图片数量'

In [127]:
input_dim = 0
for k,v in  multi_hot_column_odict.items():
    input_dim += len(v.classes_)
input_dim

4813

In [128]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(input_dim,)))
model.add(tf.keras.layers.Dense(7, activation='softmax',use_bias=True,
                                #kernel_regularizer=tf.keras.regularizers.l1(0.01)
                               ))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.binary_crossentropy, 
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [129]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 7)                 33698     
Total params: 33,698
Trainable params: 33,698
Non-trainable params: 0
_________________________________________________________________


In [130]:
model.fit(x_train, 
          y_train, 
          batch_size=64,
          epochs=20,
         validation_data = (x_vali,y_valid))

Epoch 1/20
23/23 [==============================] - 0s 5ms/step - loss: 0.3943 - binary_accuracy: 0.8572 - val_loss: 0.3763 - val_binary_accuracy: 0.8571
Epoch 2/20
23/23 [==============================] - 0s 2ms/step - loss: 0.3607 - binary_accuracy: 0.8600 - val_loss: 0.3603 - val_binary_accuracy: 0.8563
Epoch 3/20
23/23 [==============================] - 0s 2ms/step - loss: 0.3445 - binary_accuracy: 0.8632 - val_loss: 0.3496 - val_binary_accuracy: 0.8584
Epoch 4/20
23/23 [==============================] - 0s 2ms/step - loss: 0.3340 - binary_accuracy: 0.8654 - val_loss: 0.3423 - val_binary_accuracy: 0.8600
Epoch 5/20
23/23 [==============================] - 0s 2ms/step - loss: 0.3255 - binary_accuracy: 0.8678 - val_loss: 0.3364 - val_binary_accuracy: 0.8616
Epoch 6/20
23/23 [==============================] - 0s 2ms/step - loss: 0.3184 - binary_accuracy: 0.8702 - val_loss: 0.3317 - val_binary_accuracy: 0.8648
Epoch 7/20
23/23 [==============================] - 0s 2ms/step - loss: 0.31

In [132]:
model.evaluate(x_test,y_test)

14/14 [==============================] - 0s 883us/step - loss: 0.3277 - binary_accuracy: 0.8703


[0.32772091031074524, 0.8703345656394958]